<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Mean-Variance Portfolio Theory

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Portfolio Theory

Topics of interest include:

* real financial data
* log returns
* portfolio return
* portfolio risk
* possible risk-return combinations
* optimal portfolios
 * maximum Sharpe portfolio
 * minimum risk portfolio
 * efficient frontier

## Real World Data

**Instead of artificial `NumPy` or `pandas` data, real-world end-of-day time series data for financial assets is used.**

## Financial Assets

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
np.set_printoptions(suppress=True)

In [ ]:
url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw

In [ ]:
raw.info()

### Returns

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'GLD']

In [ ]:
data = raw[symbols]

In [ ]:
data.pct_change().head()  # simple daily returns

In [ ]:
rets = np.log(data / data.shift(1))  # daily log returns
rets.head()

In [ ]:
rets.hist(figsize=(10, 6), bins=35);

### Expected Return

**_Expected_** return means the return that "we expect given historical returns".

In [ ]:
rets.mean()  # expected daily return

In [ ]:
rets.mean() * 252  # expected annual return

### Volatility

In [ ]:
rets.std()  # daily volatility (= standard deviation of returns)

In [ ]:
rets.std() * math.sqrt(252)  # annualized volatility

In [ ]:
np.sqrt(rets.var() * 252)  # annualized volatility

## Portfolio Return

In [ ]:
data.shape

In [ ]:
noa = data.shape[1]
noa

In [ ]:
phi = noa * [1 / noa]
phi

In [ ]:
def port_return(phi):
    return np.dot(rets.mean() * 252, phi)

In [ ]:
port_return(phi)

## Portfolio Risk

In [ ]:
rets.cov() * 252

In [ ]:
def port_risk(phi):
    return math.sqrt(np.dot(phi, np.dot(rets.cov() * 252, phi)))

In [ ]:
port_risk(phi)

In [ ]:
rets.corr()

## Mean-Variance Portfolios

In [ ]:
p = np.random.random((500, noa))
p = (p.T / p.sum(axis=1)).T
p[:10].sum(axis=1)

In [ ]:
rr = np.array([(port_risk(phi), port_return(phi)) for phi in p])

In [ ]:
rr[:10]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr[:, 0], rr[:, 1], 'ro');

## Optimal Portfolios

In [ ]:
from scipy.optimize import minimize

### Maximum Sharpe Portfolio

In [ ]:
def sharpe(phi):
    return port_return(phi) / port_risk(phi)

In [ ]:
sharpe(phi)

In [ ]:
bnds = noa * [(0, 1)]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: phi.sum() - 1}

In [ ]:
opt = minimize(lambda phi: -sharpe(phi), phi, bounds=bnds, constraints=cons)
opt

In [ ]:
phi_ = opt['x']

In [ ]:
sharpe(phi_)  # maximum Sharpe ratio

In [ ]:
port_return(phi_)

In [ ]:
port_risk(phi_)

### Minimum Risk Portfolio

In [ ]:
opt = minimize(port_risk, phi, bounds=bnds, constraints=cons)
opt

In [ ]:
phi__ = opt['x']

In [ ]:
sharpe(phi__)

In [ ]:
port_return(phi__)

In [ ]:
port_risk(phi__)  # minimum risk

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rr[:, 0], rr[:, 1], 'ro')
plt.plot(port_risk(phi_), port_return(phi_), 'y^', ms=15,
         label='maximum Sharpe portfolio')
plt.plot(port_risk(phi__), port_return(phi__), 'bv', ms=15,
         label='minimum risk portfolio')
plt.xlabel('portfolio risk')
plt.ylabel('portfolio return')
plt.legend();

**Exercise**: Plot the MVP bullet with two assets only and the one for three assets into the same figure and compare them.

### Efficient Frontier

**Exercise**: Derive the efficient frontier in the following two ways:

* fix the portfolio target return and derive the minimum risk portfolio for the target return
* fix the portfolio target risk and derive the maximum return portfolio for the target risk

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>